In [ ]:
import datetime
import re
# 写一个分块函数，以1000为单位分块
def sens_into_block(sens,max_chars_per_block = 1000):
    """文件按5000字拆分成多个文件"""
    # 设置每个块的最大字符数
    max_chars_per_block = 1000
    # 初始化一个空字符串，用于存储当前块的字符
    current_block_ls = []
    current_block = ''
    # 存储所有块的列表
    blocks = []

    # 逐行读取文件内容
    for line in sens:
        current_block_ls.append(line)  # 积累当前块的字符
        current_block += line
        # 检查当前块的字符数是否超过5000
        if len(current_block) > max_chars_per_block:
            # 寻找最后一个完整句子的位置
            current_block_ls.pop()
            # 将完整句子作为一个块保存
            blocks.append(''.join(current_block_ls).rstrip('\n'))
            current_block_ls.clear()
            current_block_ls.append(line)
            current_block = line

    # 处理剩余的字符作为最后一个块
    if current_block:
        blocks.append(''.join(current_block_ls).rstrip('\n'))
    # 现在，blocks 列表包含了所有块，每个块的字符不超过5000且是一个完整句子
    # 可以按需处理这些块，例如写入新文件或进行其他操作
    for block in blocks:
        yield block
        
def current_time_to_timestamp():
    # 获取当前系统时间
    current_time = datetime.datetime.now()
    timestamp_milliseconds = int(current_time.timestamp() * 1000)
    return timestamp_milliseconds

def get_sign_and_asc_token(keyword):
        import execjs
        with open('baidusign.js', 'r', encoding='utf-8') as f:
                js = f.read()
                sign = execjs.compile(js).call('tl', keyword)
        with open('Acs-token.js', 'r', encoding='utf-8') as f:
                js = f.read()
                acs_token = execjs.compile(js).call('translate', keyword)
                # print(acs_token)
        return sign,acs_token

def get_token(session):
    url = 'https://fanyi.baidu.com/?aldtype=16047'
    session.headers.update({'Referer':'https://fanyi.baidu.com/?aldtype=16047'})
    response = session.get(url)
    if response.status_code != 200:
        raise ValueError(f"Failed to get a valid response, status code: {response.status_code}")
    pattern = r"window\['common'\] = {\s*token: '(.*?)',"
    token = re.findall(pattern, response.text, re.S)
    if token:
        return token[0]
    else:
        raise ValueError("Token not found in the response.")

def translate_word(keyword):
    import requests
    sign,acs_token = get_sign_and_asc_token(keyword)

    with requests.Session() as session:     
        session.headers={   
            'Host': 'fanyi.baidu.com',# cookie 需要定期更换
            'Cookie': 'BAIDUID=0CE7367CEAF67F2FB16F4A08E3CFA6FC:FG=1; BAIDUID_BFESS=0CE7367CEAF67F2FB16F4A08E3CFA6FC:FG=1; ppfuid=FOCoIC3q5fKa8fgJnwzbE67EJ49BGJeplOzf+4l4EOvDuu2RXBRv6R3A1AZMa49I27C0gDDLrJyxcIIeAeEhD8JYsoLTpBiaCXhLqvzbzmvy3SeAW17tKgNq/Xx+RgOdb8TWCFe62MVrDTY6lMf2GrfqL8c87KLF2qFER3obJGluLczhecqgremvAygsVvJeGEimjy3MrXEpSuItnI4KD6G2TVbOPDRn86IYxl35j3vHXCZJlKkExq27U3cnU4QLrqUSGMNte0pZlamiwBRRPRJsVwXkGdF24AsEQ3K5XBbh9EHAWDOg2T1ejpq0s2eF1B2GaP1HziTgKv97Jkw7/vpaEhZpob9le2b5QIEdiQdtJfhN1eLb/i/C9hcVPjDWFCMUN0p4SXVVUMsKNJv2T9/Vy/ctPNWsWYSfY9V7sGzasmgOrJ40n63OsKSOpoSLYt4idEszwDd8MfVczsvON1XPPZzT+5pNmCHs6oLo0wXQ+//wdrn6SUz7a0vEMm7QqGqBJJILGchC/ZM0axiniVRKx4R3cqVpTVNqTP1tWGnGGu/AVLS3NcPF3XemJkZyi6L0BPA661JDj0lmZIgcCHm0lGODoYWzuL7ZDizBm0d8BJIJUS1lUOPNebjg5OCjwkSq16g64gugrO/OhN+XjRMTNne43cKuMDmex1CEngB2QvyTjxXMcJvDDEe3McIycHFbZmbEY9LT3RuWsSjij5HIeKAxeCJRzKQmiJrt2NfLgbCuqJYL1ds7gVgZIml9ItCSOhy8hYtkazHuytpZ4mHgNUoL5fOnYDwL4EdXH15/q1uQn9VsBjBmLNQsYnwiX1i39zQE19TGybrzqrM1pICo4W+Id8GDvscflaNVaQHaBDBV0+6QTz5Y6CfsuieRrNsXbKicl3rMF+gCh2AJdvWihdtd/7B191wLsHTodMtFIQhggFdGBuKFzw7/79kCP/JaNQU6nPurqIqmaFT3GHOSur+fOpFAgP8tGwklbbh77U3jCUnOE2YVcxDRiJhV2rtoc3z07Ju637GxzfwnojhZ3atWLnGsfyEaq93vs4m7Mfxf61vQ9X5ee7c0vV+aosiH5dwXp+fI8U9TG15C1u0vKL6CrnOjCDWgWPw78v2hWygrbZr2pSYz8eskxGwuLX4n1pJ3jb5fvx8MhmCQ+Hd6nMsu/BGacBuznP9X6OXc/eBWeXCb1t/CFoW2eEU3Z0dgDDuLhCGtd3NzvuKxeEPnnl7/ay+EKmYuqWsJpU6QiXzUhqx78LXpdU8o6P1c; smallFlowVersion=old; RT="z=1&dm=baidu.com&si=ca0e3b4f-9578-42f9-b0f3-078069ce95d6&ss=lzqocbxl&sl=a&tt=e12&bcn=https%3A%2F%2Ffclog.baidu.com%2Flog%2Fweirwood%3Ftype%3Dperf&ld=10asm&ul=10uln&hd=10uo9"; REALTIME_TRANS_SWITCH=1; FANYI_WORD_SWITCH=1; HISTORY_SWITCH=1; SOUND_SPD_SWITCH=1; SOUND_PREFER_SWITCH=1; ab_sr=1.0.1_ZDU0ZGZiOWY5YWY3NTY3MWIwNmVjNDQyYjJiODNhYjg3NmQ1MzhiN2U1YzRlMDQyN2M2ZjkzZGRiODJkOTQzMDFjN2YwYzJkZGIxNjg1MWQ3MmRlMDk4MmZkNDI0MDg4YTJiMjk3OGYwMDg1NzI2OGRlOWRhMjM4OTljOWIwYTVjNTkwNWM5M2YxZGNhOTI3ODExZDFmMGMwZGYxYmY4NjY5OGNjYjA5NmIyOWVlMGQ1OGI5NGU5MDhlNjJkMGNmMmFmOGM2ZTQwZDBmNmY0ZTk1ODY5NmVlMzU2NzZjZjU=',        'Referer': 'https://fanyi.baidu.com/',
            'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'
        }
        url = "https://fanyi.baidu.com/v2transapi"
        params = {
            "from": "zh",
            "to": "en"
        }
        data = {
            "from": "zh",
            "to": "en",
            "query": keyword,
            "transtype": "realtime",
            "simple_means_flag": "3",
            "sign": sign,
            "token": get_token(session),
            "domain": "common",
            "ts":current_time_to_timestamp()
        }
        
        session.headers.update({'Acs-Token':acs_token})
        response = session.post(url,params=params, data=data)
        translated_sens='\n'.join([sen['dst'] for sen in response.json()['trans_result']['data']])
        # translate_word = response.json()['trans_result']['data'][0]['dst']
        return translated_sens


In [ ]:
if __name__ == '__main__':
    with open(r'D:\translate_clean\新建文件夹\input\0.txt', 'r', encoding='utf-8') as f:
        sens = f.readlines()
    for block in sens_into_block(sens):
        print(translate_word(block))